<a href="https://colab.research.google.com/github/pginjupalli/Verizon-EnergyPriceForecasting/blob/main/Verizon_2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Understanding the Data**

Here, we will look at the dataset and note the features, their correlations with the label, whether or not there's missing data, and come up with ideas clearn and prepare the data for modeling.

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/BTT Verizon 2/electricity_prices.csv', 'r') as f:
  df = pd.read_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(df.isnull().sum()) # columns with null values
df[df['customers'].notnull()] # rows where 'customer' is not null

year                    0
month                   0
stateDescription        0
sectorName              0
customers           26040
price                   0
revenue                 0
sales                   0
dtype: int64


,year,month,stateDescription,sectorName,customers,price,revenue,sales
26040,2008,1,Washington,all sectors,3145488.0,6.64,574.73235,8658.35972
26041,2008,1,Rhode Island,transportation,0.0,0.00,0.00000,0.00000
26042,2008,1,South Carolina,transportation,0.0,0.00,0.00000,0.00000
26043,2008,1,Massachusetts,industrial,14142.0,13.18,100.40468,761.89017
26044,2008,1,Massachusetts,residential,2611383.0,16.81,315.43624,1876.02831
...,...,...,...,...,...,...,...,...
85865,2024,1,Arkansas,all sectors,1717720.0,9.63,442.98773,4598.63147
85866,2024,1,Arkansas,commercial,208669.0,10.26,97.79467,953.02154
85867,2024,1,Arkansas,industrial,34951.0,7.08,109.92656,1553.02838
85868,2024,1,Arkansas,residential,1474098.0,11.24,235.26399,2092.56172


As this above code shows, the 'customers' feature is missing data from a third of the examples. This is too much data to throw away, so we must replace the null values with actual data. This can be done by replacing null values with values for mean, median, mode, etc.

We analyze these values for the 'customers' feature and see which value works best for replacement of null values.

In [ ]:
df_with_customers = df[df['customers'].notnull()] # rows where 'customer' is not null

print("Customers Mean: " + str(df_with_customers['customers'].mean()))
print("Customers Median: " + str(df_with_customers['customers'].median()))
print("Customers Mode: " + str(df_with_customers['customers'].mode()))

df_with_customers.select_dtypes(['int64', 'float64']).corr()

Customers Mean: 2916013.4194216947
Customers Median: 299754.0
Customers Mode: 0    0.0
Name: customers, dtype: float64


,year,month,customers,price,revenue,sales
year,1.000000,-0.014991,0.009542,0.139038,0.023120,0.004143
month,-0.014991,1.000000,0.000577,0.016232,0.005551,0.002567
customers,0.009542,0.000577,1.000000,0.083902,0.909009,0.876248
price,0.139038,0.016232,0.083902,1.000000,0.066214,0.026643
revenue,0.023120,0.005551,0.909009,0.066214,1.000000,0.987114
sales,0.004143,0.002567,0.876248,0.026643,0.987114,1.000000


This is the current statistical values for the customer's column and its correlation with others. Our full dataset should match this one as closely as possible.

Let's place the null values with the values for mean, median, and mode, and see which one closely resembles this correlation data.

In [ ]:
mean = int(df_with_customers['customers'].mean())

df_with_mean = df.copy()
df_with_mean['customers'] = df['customers'].fillna(mean)

print("Correlations with the mean as a replacement")
df_with_mean.select_dtypes(['int64', 'float64']).corr()

Correlations with the mean as a replacement


,year,month,customers,price,revenue,sales
year,1.000000,-0.010297,0.005552,0.261512,0.045368,0.008315
month,-0.010297,1.000000,0.000488,0.019868,0.006237,0.002925
customers,0.005552,0.000488,1.000000,0.074038,0.821477,0.743986
price,0.261512,0.019868,0.074038,1.000000,0.074662,0.028550
revenue,0.045368,0.006237,0.821477,0.074662,1.000000,0.979835
sales,0.008315,0.002925,0.743986,0.028550,0.979835,1.000000


In [ ]:
median = int(df_with_customers['customers'].median())

df_with_median = df.copy()
df_with_median['customers'] = df['customers'].fillna(median)

print("Correlations with the median as a replacement")
df_with_median.select_dtypes(['int64', 'float64']).corr()

Correlations with the median as a replacement


,year,month,customers,price,revenue,sales
year,1.000000,-0.010297,0.100436,0.261512,0.045368,0.008315
month,-0.010297,1.000000,0.000029,0.019868,0.006237,0.002925
customers,0.100436,0.000029,1.000000,0.100938,0.820243,0.739499
price,0.261512,0.019868,0.100938,1.000000,0.074662,0.028550
revenue,0.045368,0.006237,0.820243,0.074662,1.000000,0.979835
sales,0.008315,0.002925,0.739499,0.028550,0.979835,1.000000


In [59]:
mode = int(df_with_customers['customers'].mode()[0])

df_with_mode = df.copy()
df_with_mode['customers'] = df['customers'].fillna(mode)

print("Correlations with the mode as a replacement")
df_with_mode.select_dtypes(['int64', 'float64']).corr()

Correlations with the mode as a replacement


,year,month,customers,price,revenue,sales
year,1.000000,-0.010297,0.111121,0.261512,0.045368,0.008315
month,-0.010297,1.000000,-0.000024,0.019868,0.006237,0.002925
customers,0.111121,-0.000024,1.000000,0.103902,0.819365,0.738324
price,0.261512,0.019868,0.103902,1.000000,0.074662,0.028550
revenue,0.045368,0.006237,0.819365,0.074662,1.000000,0.979835
sales,0.008315,0.002925,0.738324,0.028550,0.979835,1.000000


From these 3 ways of replacement, replacing with the **median** had the new correlations resemble the original, expected correlations the best. So, we can replace the null values in 'customers' with the mode moving forward.

In [61]:
df['customers'] = df['customers'].fillna(median) # Replacing null values in customers with the mode
df

,year,month,stateDescription,sectorName,customers,price,revenue,sales
0,2001,1,Wyoming,all sectors,0.0,4.31,48.12840,1116.17208
1,2001,1,Wyoming,commercial,299754.0,5.13,12.67978,247.08691
2,2001,1,Wyoming,industrial,299754.0,3.26,19.60858,602.30484
3,2001,1,Wyoming,other,299754.0,4.75,0.76868,16.17442
4,2001,1,Wyoming,residential,299754.0,6.01,15.07136,250.60591
...,...,...,...,...,...,...,...,...
85865,2024,1,Arkansas,all sectors,1717720.0,9.63,442.98773,4598.63147
85866,2024,1,Arkansas,commercial,208669.0,10.26,97.79467,953.02154
85867,2024,1,Arkansas,industrial,34951.0,7.08,109.92656,1553.02838
85868,2024,1,Arkansas,residential,1474098.0,11.24,235.26399,2092.56172


# **Part 2: Analyzing the Data**

Here, we can analyze the data through statistical methods and visualize any trends.

Since our dataset shows prices on a timeline, there is no use in finding the mean of the 'price' column or others like it. Instead, we can analyze the trends through out the timeline with plots.

We can plot the revenue, sales, and prices for a each state, using the data from all sectors.